In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchattacks
import requests
import os
from torchvision import models

In [59]:
import requests
import torch
import torch.nn as nn
import numpy as np
import json
import io
import sys
import base64
from torch.utils.data import Dataset
from typing import Tuple
import pickle
import os

class TaskDataset(Dataset):
    def __init__(self):
        self.ids = []
        self.imgs = []  # Assuming these are PIL images
        self.labels = []
        # Define the transformation: Convert image to RGB and then to a tensor
        self.transform = transforms.Compose([
            transforms.Lambda(lambda img: img.convert('RGB')),
            transforms.ToTensor(),
        ])

    def __getitem__(self, index) -> Tuple[torch.Tensor, int]:
        img = self.imgs[index]
        if self.transform is not None:
            img = img.convert('RGB')
            img = transforms.ToTensor()(img)
        label = self.labels[index]
        return img, label

    def __len__(self):
        return len(self.imgs)


data = TaskDataset()

In [60]:
data: TaskDataset = torch.load("./Train.pt")

In [61]:
data[0]

(<PIL.Image.Image image mode=RGB size=32x32>, 8)

In [56]:
# Assuming `data[0][0]` is the PIL image you want to convert
image = data[0][0]

# Define the transformation
transform = transforms.Compose([
    transforms.ToTensor()  # Converts PIL Image to Tensor
])

# Apply the transformation
image_tensor = transform(image)

In [58]:
image_tensor.shape

torch.Size([3, 32, 32])

In [5]:
os.makedirs("out/models", exist_ok=True)


In [12]:
# Create a dummy model
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.weight.shape[1], 10)

In [13]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# DataLoader
data_loader = DataLoader(data, batch_size=32, shuffle=True)

In [14]:
for inputs, labels in data_loader:
    print(inputs.shape)

TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>

In [ ]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, labels in data_loader:
        # Convert tensors to require_grad   
        adversarial_inputs = inputs.clone().detach().requires_grad_(True)
        labels = labels.clone().detach()
                
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(adversarial_inputs)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        loss.backward()
        optimizer.step()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

In [ ]:
def fgsm_attack(image, epsilon, data_grad):
    sign_data_grad = data_grad.sign()
    perturbed_image = image + epsilon * sign_data_grad
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

def pgd_attack(model, images, labels, eps=0.03, alpha=0.01, iters=40):
    original_images = images.clone()
    for _ in range(iters):
        images.requires_grad = True
        outputs = model(images)
        model.zero_grad()
        loss = F.nll_loss(outputs, labels)
        loss.backward()
        adv_images = images + alpha * images.grad.sign()
        eta = torch.clamp(adv_images - original_images, min=-eps, max=eps)
        images = torch.clamp(original_images + eta, min=0, max=1).detach_()
    return images

In [ ]:



#### SUBMISSION ####


torch.save(model.state_dict(), "out/models/dummy_submission.pt")

#### Tests ####
# (these are being ran on the eval endpoint for every submission)

allowed_models = {
    "resnet18": models.resnet18,
    "resnet34": models.resnet34,
    "resnet50": models.resnet50,
}
with open("out/models/dummy_submission.pt", "rb") as f:
    try:
        model: torch.nn.Module = allowed_models["resnet18"](weights=None)
        model.fc = torch.nn.Linear(model.fc.weight.shape[1], 10)
    except Exception as e:
        raise Exception(
            f"Invalid model class, {e=}, only {allowed_models.keys()} are allowed",
        )
    try:
        state_dict = torch.load(f, map_location=torch.device("cpu"))
        model.load_state_dict(state_dict, strict=True)
        model.eval()
        out = model(torch.randn(1, 3, 32, 32))
    except Exception as e:
        raise Exception(f"Invalid model, {e=}")

    assert out.shape == (1, 10), "Invalid output shape"


# Send the model to the server
response = requests.post("http://34.71.138.79:9090/robustness", files={"file": open("out/models/dummy_submission.pt", "rb")}, headers={"token": "TOKEN", "model-name": "resnet18"})

# Should be 400, the clean accuracy is too low
print(response.json())